<h1><center>Physique numérique (PHY-3500) - TP2</center></h1>
<hr>
<br><br>

**Membres de l'équipe**
| Nom | NI |
| --- | :---: |
| Maxime Tousignant-Tremblay | 536 772 369 |
| Philippe Morin | 536 776 382 |
| Emilie Carré Smith| 111 235 839 |


In [1]:
import numpy as np
from matplotlib import pyplot as plt


%matplotlib inline
%config InlineBackend.figure_formats = "svg",
plt.style.use("LabReport.mplstyle")

# Numéro 1

A) *À l’aide des équations (2.1.2) et (2.1.3), démontrez que les matrices de réflexion $\boldsymbol{Q_i}$ sont orthogonales.*

B) *Démontrez l’équation (2.1.5) et que la matrice $\boldsymbol{Q}$ est orthogonale.*

C) *Implémentez la fonction householder_qr qui prend en argument une matrice $\boldsymbol{A}$ et qui retourne les matrices $\boldsymbol{Q}$ et $\boldsymbol{R}$ obtenues par la méthode de Householder.*

In [2]:
def householder_qr(A: np.ndarray[np.float64]) -> tuple[np.ndarray[np.float64]]:
    """Calcule la décomposition QR de la matrice A par la méthode de Householder.

    Paramètres
    ----------
    A : numpy.ndarray
        Matrice d'entrée.

    Retours
    -------
    Q : numpy.ndarray
        Matrice orthogonale.
    R : numpy.ndarray
        Matrice triangulaire supérieure.

    """
    m, n = A.shape
    Q = np.eye(m)

    for j in range(min(m - 1, n)):
        x = A[j:, j]
        x_norm = np.linalg.norm(x)
        e = np.zeros_like(x)
        e[0] = x_norm
        v = np.sign(x[0]) * x_norm * e + x
        v /= np.linalg.norm(v)

        # Appliquer la transformation de Householder sur A
        A[j:, :] -= 2 * np.outer(v, (v @ A[j:, :]))

        # Appliquer la transformation de Householder sur Q
        Q[j:, :] -= 2 * np.outer(v, (v @ Q[j:, :]))

    # Tronquer pour obtenir la matrice triangulaire supérieure
    R = np.triu(A[:n, :])
    return Q.T, R

D) *À l’aide d’une matrice de dimension 4 × 3 de votre choix, testez votre fonction householder_qr et comparez les résultats obtenus avec ceux obtenus à l’aide de la fonction* `numpy.linalg.qr`. *Les matrices sont-elles exactement les mêmes ? Si non, est-ce un problème ?*

In [3]:
# Tester la fonction avec une matrice aléatoire 4 x 3
A = np.random.rand(4, 3)
Q, R = householder_qr(A)
print("Décomposition QR de householder_qr :")
print(f"Q :\n{Q}")
print(f"R :\n{R}")

# Comparaison avec la décomposition QR de numpy
Q_np, R_np = np.linalg.qr(A)
print("\nDécomposition QR de Numpy :")
print(f"Q :\n{Q_np}")
print(f"R :\n{R_np}")

Décomposition QR de householder_qr :
Q :
[[-0.5246208   0.72895736  0.3384927   0.28074343]
 [-0.65075956 -0.01335285 -0.63589641 -0.41469198]
 [-0.44890893 -0.59685948  0.03900251  0.66386621]
 [-0.31585721 -0.33496513  0.69248628 -0.55542357]]
R :
[[-1.14258974 -0.70792759 -1.42756818]
 [ 0.          0.48502323 -0.13446888]
 [ 0.          0.          0.43101482]]

Décomposition QR de Numpy :
Q :
[[-0.99647912  0.082099    0.00506979]
 [-0.0717892  -0.93940906  0.1609431 ]
 [-0.03333545 -0.27193061 -0.87569942]
 [-0.02764818 -0.19189612  0.45521659]]
R :
[[ 1.14662688  0.66371207  1.42527469]
 [ 0.         -0.56702731 -0.05639656]
 [ 0.          0.         -0.5289384 ]]


Non, les matrices $\boldsymbol{Q}$ et $\boldsymbol{R}$ obtenues à partir de la décomposition QR de householder_qr et de la décomposition QR de Numpy ne sont pas exactement les mêmes. Cependant, cela n'est pas nécessairement un problème. Il est courant que les implémentations de décomposition QR puissent donner des résultats légèrement différents en raison de variations dans les méthodes numériques utilisées ou dans la manière dont les calculs sont effectués. Cela peut conduire à des différences dans les valeurs numériques, même si les matrices résultantes sont équivalentes en termes mathématiques.

Dans la plupart des cas, ces petites différences ne sont pas problématiques et peuvent être dues à des erreurs d'arrondi ou à d'autres considérations numériques. Tant que les résultats sont proches et que les propriétés fondamentales de la décomposition QR sont respectées (comme l'orthogonalité de $\boldsymbol{Q}$ et la triangularité de $\boldsymbol{R}$), les résultats sont considérés comme valides.

En résumé, bien que les matrices ne soient pas exactement les mêmes, cela ne pose généralement pas de problème tant que les résultats sont numériquement stables et que les propriétés essentielles de la décomposition QR sont respectées.

E) *À l’aide de la matrice utilisée en D, illustrez comment la multiplication successive des matrices $\boldsymbol{Q_i}$ triangularise progressivement la matrice $\boldsymbol{A}$. Dans l’élan, assurez-vous que les matrices $\boldsymbol{Q}$ et $\boldsymbol{R}$ obtenues sont bien orthogonale et triangulaire supérieure, respectivement.*

In [4]:
# Affichage de la matrice A
print(f"Matrice A :\n{A}")

# Affichage des matrices Q et R
print(f"\nMatrice Q :\n{Q}")
print(f"\nMatrice R :\n{R}")

# Vérification de l'orthogonalité de Q
print("\nVérification de l'orthogonalité de Q :")
print(f"Q X Q^T :\n{Q @ Q.T}")

# Vérification de la triangularité de R
print("\nVérification de la triangularité de R :")
print(f"R :\n{R}")

Matrice A :
[[-1.14258974 -0.70792759 -1.42756818]
 [-0.08231543  0.48502323 -0.13446888]
 [-0.03822332  0.13206694  0.43101482]
 [-0.03170215  0.09045991 -0.26936551]]

Matrice Q :
[[-0.5246208   0.72895736  0.3384927   0.28074343]
 [-0.65075956 -0.01335285 -0.63589641 -0.41469198]
 [-0.44890893 -0.59685948  0.03900251  0.66386621]
 [-0.31585721 -0.33496513  0.69248628 -0.55542357]]

Matrice R :
[[-1.14258974 -0.70792759 -1.42756818]
 [ 0.          0.48502323 -0.13446888]
 [ 0.          0.          0.43101482]]

Vérification de l'orthogonalité de Q :
Q X Q^T :
[[ 1.00000000e+00 -8.15513246e-17  1.40501187e-16 -1.21485541e-17]
 [-8.15513246e-17  1.00000000e+00  2.60681487e-16  1.04366052e-17]
 [ 1.40501187e-16  2.60681487e-16  1.00000000e+00 -2.75038159e-16]
 [-1.21485541e-17  1.04366052e-17 -2.75038159e-16  1.00000000e+00]]

Vérification de la triangularité de R :
R :
[[-1.14258974 -0.70792759 -1.42756818]
 [ 0.          0.48502323 -0.13446888]
 [ 0.          0.          0.43101482]]


# Numéro 2

A) *Modifiez votre code de décomposition QR pour qu’il retourne la décomposition QR réduite de la matrice d’entrée lorsque l’argument additionnel reduite=True lui est passé.*

In [5]:
def householder_qr(
    A: np.ndarray[np.float64],
    reduite: bool = False,
) -> tuple[np.ndarray[np.float64]]:
    """Calcule la décomposition QR de la matrice A par la méthode de Householder.

    Paramètres
    ----------
    A
        Matrice d'entrée.
    reduite
        Indique si la décomposition QR réduite doit être retournée (par défaut False).

    Retours
    -------
    Q
        Matrice orthogonale.
    R
        Matrice triangulaire supérieure (ou inférieure si réduite est True).

    """
    m, n = A.shape
    Q = np.eye(m)

    for j in range(min(m - 1, n)):
        x = A[j:, j]
        x_norm = np.linalg.norm(x)
        e = np.zeros_like(x)
        e[0] = x_norm
        v = np.sign(x[0]) * x_norm * e + x
        v /= np.linalg.norm(v)

        # Appliquer la transformation de Householder sur A
        A[j:, :] -= 2 * np.outer(v, (v @ A[j:, :]))

        # Appliquer la transformation de Householder sur Q
        Q[j:, :] -= 2 * np.outer(v, (v @ Q[j:, :]))

    if reduite:
        # Utiliser np.tril pour obtenir la matrice triangulaire inférieure (réduite)
        R = np.tril(A[:, :n])
    else:
        # Utiliser np.triu pour obtenir la matrice triangulaire supérieure
        R = np.triu(A[:, :n])

    return Q.T, R

B) *Utilisez votre code pour résoudre approximativement l’équation (2.2.4). Vous utiliserez les données fournies dans le fichier bataille_navale_equipeXX.csv où vous remplacerez XX par votre numéro d’équipe dans la boîte de dépôt sur MonPortail.*

C) *Tracez les données (cercles noirs) et la solution estimée de la trajectoire (ligne pleine de la couleur de votre choix) donnée par l’équation (2.2.3).*

D) *Obtenez la position d’impact du projectile (à $y = 0$) en résolvant l’équation quadratique (2.2.3) pour $x$ à l’aide d’une implémentation personnelle de la méthode de la bissection. Comparez votre solution avec celle obtenue en résolvant cette même équation analytiquement. Considérant que votre embarcation se situe à la position ($x$, $y$) = (0, 0), quelle est la distance horizontale vous séparant du point d’impact ?*

# Numéro 3

A) *Trouvez analytiquement toutes les solutions de l’équation cubique (2.3.3).*

B) *Démontrez laquelle des solutions obtenues précédemment (ou une combinaison de celles-ci) correspond à $u$, soit la solution recherchée de l’équation (2.3.3).*

C) *Tracez $R_\infty$ en fonction de $T$ et identifiez tout changement qualitatif de $R_\infty$. Comment interprétez-vous ce changement (ou cette absence de changement) ?*

D) *Résolvez numériquement l’équation (2.3.3) à l’aide d’implémentations personnelles de la méthode par relaxation et de la méthode de Newton-Raphson pour 20 valeurs de $T$ uniformément distribuées dans l’intervalle [0, 1]. Illustrez vos résultats à l’aide d’un graphique comparant les solutions analytiques (indiquées avec des lignes de couleurs distinctes) aux solutions numériques obtenues avec différentes valeurs initiales de l’algorithme (symboles ; choisissez bien vos symboles pour que vos solutions soient bien visibles). Arrivez-vous à obtenir les trois solutions identifiées en $\textbf{A}$ ? Pourquoi ? Considérez tracer l’équation (2.3.3) de même que la dérivée de $f(u)$ en fonction de $u$ et ce, pour quelques valeurs de $T$, pour appuyer vos conclusions. Vous pouvez aussi tracer les itérations successives de chacune des méthodes afin d’illustrer la manière dont elles convergent vers l’une ou l’autre des solutions (ex. : tracer un diagramme en toile d’araignée).*